# PCA QAOA

In this notebook we explore dimensionality reduction methods to train QAOA angles [1].

### References

[1] O. Parry and P. McMinn, "QAOA-PCA: Enhancing Efficiency in the Quantum Approximate Optimization Algorithm via Principal Component Analysis" [arXiv:2504.16755](https://arxiv.org/abs/2504.16755)


In [1]:
import networkx as nx

from qaoa_training_pipeline.training import TQATrainer, ScipyTrainer
from qaoa_training_pipeline.evaluation import StatevectorEvaluator
from qaoa_training_pipeline.utils.graph_utils import graph_to_operator

First, we will generate a small database of a few good angles for some problems that we define.

In [2]:
trainer1 = TQATrainer(StatevectorEvaluator())
trainer2 = ScipyTrainer(StatevectorEvaluator())

In [3]:
num_instances = 5
qaoa_depth = 4
num_nodes = 8

instances, results = [], []

for idx in range(num_instances):
    instances.append(nx.erdos_renyi_graph(n=num_nodes, p=0.5, seed=idx))
    cost_op = graph_to_operator(instances[-1], pre_factor=-0.5)

    result1 = trainer1.train(cost_op, reps=qaoa_depth)
    result2 = trainer2.train(cost_op, params0=result1["optimized_params"])

    results.append(result2["optimized_params"])

In [4]:
from qaoa_training_pipeline.training import QAOAPCA
from qaoa_training_pipeline.training.data_loading import TrivialDataLoader

In [5]:
data_loader = TrivialDataLoader(results)
pca_trainer = QAOAPCA(data_loader, num_components=2, evaluator=StatevectorEvaluator())

In [6]:
new_graph = nx.erdos_renyi_graph(n=num_nodes, p=0.5, seed=2*num_instances)
new_cost_op = graph_to_operator(new_graph, pre_factor=-0.5)

In [7]:
pca_trainer.train(new_cost_op, params0=[0, 0])

ParamResult(data={'system_info': {'python_version': '3.12.10', 'system': 'Windows', 'processor': 'Intel64 Family 6 Model 170 Stepping 4, GenuineIntel', 'platform': 'Windows-11-10.0.26200-SP0', 'qaoa_training_pipeline_version': 16}, 'optimized_params': [0.12140280308636536, 0.5719567090952555], 'optimized_qaoa_angles': [1.419472656444762, 0.3696915904547534, 0.3193719510201666, 0.18525383081727737, -0.38691959987652835, 0.8051828100802466, 0.6945574114276034, 0.8936687023628144], 'train_duration': 0.2246379852294922, 'energy': 2.8550132701331363, 'trainer': {'trainer_name': 'QAOAPCA', 'evaluator': {'name': 'StatevectorEvaluator', 'statevector_init_args': {}}, 'method': 'COBYLA', 'num_components': 2, 'data_loader': {'function_name': 'TrivialDataLoader', 'data': [[1.3863624991868284, 0.41152550963468765, 0.33516647631794577, 0.17597137922393555, -0.3124016341974195, 0.7040582494335497, 0.8708933843297055, 1.1154345227846003], [1.4373830917159593, 0.35311185257475014, 0.294969945206751, 0.